## FIRST MINE SOME TEXT

In [189]:
import requests
import os
from pprint import pprint
from bs4 import BeautifulSoup
from IPython.display import clear_output
import numpy as np
import pandas as pd
import time
import pickle
import copy
import tqdm


In [24]:
page = requests.get("https://www.nytimes.com/2020/07/31/business/trump-financial-disclosure.html")
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
apikey = '' #SET your nytimes apikey


section = "science"
year=2020
for month in [1,2,3,4,5,6,7,8]:
  print(f"month {month}:")
  query_url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apikey}"

  r = requests.get(query_url)

  results=[]
  temp=copy.deepcopy(r.json()["response"]["docs"])
  for i,web in  enumerate(tqdm.tqdm(r.json()["response"]["docs"])):
    address=web["web_url"]
    page = requests.get(address)
    soup = BeautifulSoup(page.content, 'html.parser')
    texts=soup.find_all("div",class_="css-s99gbd StoryBodyCompanionColumn")
    text=""
    for item in texts:
      text=text+(item.text)
    temp[i]["full_text"]=text
  results.extend(temp)

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
# file = open("/content/gdrive/MyDrive/articles_2020_8.pkl",'rb')
# results = pickle.load(file)
# file.close()

In [34]:
text=""
for i in range(len(results)):
  text= text+ results[i]["full_text"]+ "\n"

#### you can replace your own text

## Train the word2vec

In [37]:
with open("temp.txt","w+") as f:
  f.write(text)

In [5]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [6]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [7]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [8]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [9]:
# text_ds = tf.data.TextLineDataset("temp.txt").filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [10]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 8192
sequence_length = 30

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [11]:
vectorize_layer.adapt(text_ds.batch(1024))

In [203]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:40])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your', 'his', 'this', 'but', 'he', 'have', 'as', 'thou', 'him', 'so', 'what', 'thy', 'will', 'no', 'by', 'all', 'king', 'we', 'shall', 'her', 'if']


In [204]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [205]:
sequences = list(text_vector_ds.as_numpy_iterator())

In [206]:
vectorize_layer.vocabulary_size()

8192

In [207]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=3,
    num_ns=6,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:23<00:00, 1379.79it/s]




targets.shape: (114704,)
contexts.shape: (114704, 7)
labels.shape: (114704, 7)


In [208]:
BATCH_SIZE = 1024
num_ns = 4
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 7), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 7), dtype=tf.int64, name=None))>


In [209]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [210]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [211]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [212]:
word2vec.fit(dataset, epochs=35,)

Epoch 1/35
112/112 [==============================] - 9s 67ms/step - loss: 1.9437 - accuracy: 0.1707
Epoch 2/35
112/112 [==============================] - 4s 34ms/step - loss: 1.8989 - accuracy: 0.4082
Epoch 3/35
112/112 [==============================] - 4s 32ms/step - loss: 1.7998 - accuracy: 0.3851
Epoch 4/35
112/112 [==============================] - 4s 32ms/step - loss: 1.6843 - accuracy: 0.4252
Epoch 5/35
112/112 [==============================] - 4s 32ms/step - loss: 1.5630 - accuracy: 0.4819
Epoch 6/35
112/112 [==============================] - 4s 32ms/step - loss: 1.4418 - accuracy: 0.5390
Epoch 7/35
112/112 [==============================] - 4s 32ms/step - loss: 1.3254 - accuracy: 0.5938
Epoch 8/35
112/112 [==============================] - 4s 32ms/step - loss: 1.2166 - accuracy: 0.6424
Epoch 9/35
112/112 [==============================] - 4s 31ms/step - loss: 1.1160 - accuracy: 0.6834
Epoch 10/35
112/112 [==============================] - 3s 31ms/step - loss: 1.0240 - accura

In [213]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [214]:
def get_embedding(word):
  return weights[vectorize_layer(word)[0].numpy()]

In [215]:
words_vec=weights
words=np.array(vectorize_layer.get_vocabulary())
words_length=np.sqrt((words_vec**2).sum(axis=1))

In [235]:
def get_k_closest(word_vec,k=32):
  if type(word_vec)==str:
    word_vec=get_embedding(word_vec)
  word_length=np.sqrt((word_vec**2).sum())
  return words[((word_vec*words_vec).sum(axis=1)/words_length/word_length*-1).argsort()[:k]]

In [236]:
man=get_embedding("man")
woman=get_embedding("woman")
king=get_embedding("king")

get_k_closest(man)

array(['uncontrolld', 'warder', 'pities', 'taketh', 'doubled',
       'unpossessd', 'wins', 'laughing', 'gyves', 'quondam', 'spoild',
       'infer', 'mass', 'softly', 'blubbering', 'spited', 'shelter',
       'victor', 'sorts', 'declining', 'laboured', 'undergo',
       'unfeignedly', 'stoppd', 'untouchd', 'confessed', 'veriest',
       'unsafe', 'unrolled', 'clerk', 'unwillingness', 'whipped'],
      dtype='<U18')

### TEST GENISM

In [1]:
import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')
wv = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [4]:
wv.most_similar(["king","woman"],["man"])

[('queen', 0.7698541283607483),
 ('monarch', 0.6843380928039551),
 ('throne', 0.6755735874176025),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534753799438),
 ('prince', 0.6517034769058228),
 ('elizabeth', 0.6464517712593079),
 ('mother', 0.6311717629432678),
 ('emperor', 0.6106470823287964),
 ('wife', 0.6098655462265015)]

In [64]:
words2=[]
words_vec=[]
for word in words:
  try:
    words_vec.append(wv[word])
    words2.append(word)
  except:
    pass
words_vec=np.array(words_vec)
words=np.array(words2)

In [65]:
def get_embedding(word):
  return wv[word]

In [66]:
words_length=np.sqrt((words_vec**2).sum(axis=1))

In [68]:
def get_k_closest(word_vec,k=32):
  if type(word_vec)==str:
    word_vec=get_embedding(word_vec)
  word_length=np.sqrt((word_vec**2).sum())
  return words[((word_vec*words_vec).sum(axis=1)/words_length/word_length*-1).argsort()[:k]]

In [70]:
man=get_embedding("man")
woman=get_embedding("woman")
king=get_embedding("king")

get_k_closest(king-man+woman)

array(['king', 'queen', 'monarch', 'throne', 'daughter', 'prince',
       'princess', 'mother', 'elizabeth', 'father', 'wife', 'son',
       'sister', 'widow', 'crown', 'emperor', 'cousin', 'lady',
       'margaret', 'married', 'kingdom', 'marriage', 'brother', 'marry',
       'birth', 'eldest', 'niece', 'death', 'husband', 'consort',
       'nephew', 'edward'], dtype='<U15')